In [ ]:
model = Sequential()
model.add(Dense(6, activation='linear'))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

X_train = np.random.normal(0,1,(10000,4))

a = np.reshape(2*X_train[:,0] + 1.5*X_train[:,1] + 0.6*X_train[:,2] + 1.2*X_train[:,3], (10000,1))
b = np.reshape(1*X_train[:,0] + 0.5*X_train[:,1] + 1.6*X_train[:,2] + 2.2*X_train[:,3], (10000,1))
y_train = np.concatenate((a,b), axis=1)

In [ ]:
from keras.callbacks import LambdaCallback

#Print weights without biais weigths after each epoch
print_weights_aftereachepoch = LambdaCallback(on_epoch_end=lambda epoch, logs: print(model.layers[0].get_weights()))
model.fit(X_train, y_train,  verbose=1, epochs=5, batch_size=1000, callbacks = [print_weights_aftereachepoch])

#Print weights without biais weigths after each batch
print_weights_aftereachbatch = LambdaCallback(on_batch_end=lambda batch, logs: print(model.layers[0].get_weights()))
model.fit(X_train, y_train,  verbose=1, epochs=5, batch_size=1000, callbacks = [print_weights_aftereachbatch])

In [ ]:
from keras.callbacks import Callback

#Save model weigths each N epochs (use load.weights for another model after)
class WeightsSaver(Callback):
    def __init__(self, N):
        self.N = N
        self.epoch = 0

    def on_epoch_end(self, epoch, logs={}):
        if self.epoch % self.N == 0:
            name = 'weights%08d.h5' % self.epoch
            self.model.save_weights(name)
        self.epoch += 1
        
model.fit(X_train, y_train, epochs=5, batch_size=1000, callbacks=[WeightsSaver(1)])


#Save model weigths each N batches (use load.weights for another model after)
class WeightsSaver(Callback):
    def __init__(self, N):
        self.N = N
        self.batch = 0

    def on_batch_end(self, batch, logs={}):
        if self.batch % self.N == 0:
            name = 'weights%08d.h5' % self.batch
            self.model.save_weights(name)
        self.batch += 1
        
model.fit(X_train, y_train, epochs=5, batch_size=1000, callbacks=[WeightsSaver(1)])

In [ ]:
from keras.callbacks import Callback

#Function to have weights saved in a variable each N epochs
class WeightsSaver(Callback):
    def __init__(self, N):
        self.N = N
        self.epoch = 0
        self.epoch_weights= {}

    def on_epoch_end(self, epoch, logs={}):
        if self.epoch % self.N == 0:
            self.epoch_weights[self.epoch] = self.model.get_weights()
        self.epoch += 1

checkpoint = WeightsSaver(1)
model.fit(X_train, y_train, epochs=5, batch_size=1000, callbacks=[checkpoint])
#example : checkpoint.epoch_weights[0] contains the weights at the end of the first epoch

#Function to have weights saved in a variable each N epochs
class WeightsSaver(Callback):
    def __init__(self, N):
        self.N = N
        self.batch = 0
        self.batch_weights= {}

    def on_batch_end(self, batch, logs={}):
        if self.batch % self.N == 0:
            self.batch_weights[self.batch] = self.model.get_weights()
        self.batch += 1

checkpoint = WeightsSaver(1)
model.fit(X_train, y_train, epochs=5, batch_size=1000, callbacks=[checkpoint])
#example : checkpoint.batch_weights][11] contains the weihts at the end of the 12th batch